In [5]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import LeaveOneOut, KFold, cross_val_predict,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline,make_pipeline
from xgboost import XGBRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, ConstantKernel as C,DotProduct, RationalQuadratic
from sklearn.neural_network import MLPRegressor
import shap
import warnings
warnings.filterwarnings("ignore")

In [6]:
# 初始准备工作
file_path = "Final_Features_12.7.csv" 
df = pd.read_csv(file_path)

## 定义特征组
### 力学特征
features_mech = [
    'X1_SoftSeg', 'X2_DA_Content', 'X3_HardSeg', 'X4_R_Ratio', 'X5_Crosslink', 
    'DA_strategy', 'cross_class',
    'Polyol_Type', 'Polyol_Mw_Score', 'Soft_Cryst',
    'Iso_Type', 'Hard_Symmetry', 
    'Interact_Cryst_Content', 'Synergy_Feature','Soft_Mw'
]
### 愈合特征
features_heal = features_mech + ['healing_temperature', 'healing_time']

### 新增特征
df['Constraint_Factor'] = df['X3_HardSeg'] / df['Soft_Mw']
features_final = features_heal + ['Constraint_Factor']

print("新特征添加完毕！前 5 行预览：")
print(df[['sample_id', 'Soft_Mw', 'Constraint_Factor']].head())

## 二次清洗
def get_clean_data(target_col, feature_cols, df):
    # 剔除空值
    data = df.dropna(subset=[target_col] + feature_cols).copy()
    X = data[feature_cols].values
    y = data[target_col].values
    return X, y

## 定义评估函数 
def evaluate_model(model, X, y, task_name="Task"):
    # 使用 LOOCV (留一法) 适合小样本
    cv = LeaveOneOut() 
    
    # 预测
    # 注意：对于 SVR/ANN 等对尺度敏感的模型，我们在 Pipeline 里已经封装了 Scaler，
    # 但为了保险，这里建议外部也可以统一 Scale，或者依赖模型内部的 Pipeline
    y_pred = cross_val_predict(model, X, y, cv=cv, n_jobs=-1)
    
    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    return r2, rmse, y_pred

print("环境设置完毕，特征组已定义。")
print("数据概览 (前5行):")
print(df[features_final].head())
print(f"\n有效总样本数: {len(df)}")

新特征添加完毕！前 5 行预览：
       sample_id  Soft_Mw  Constraint_Factor
0       DAG-PU-1   1000.0           0.000335
1       DAG-PU-2   1000.0           0.000365
2       DAG-PU-3   1000.0           0.000392
3  5%Cu-BQDU-CPU   1000.0           0.000425
4      DAG-LPU-3   1000.0           0.000251
环境设置完毕，特征组已定义。
数据概览 (前5行):
   X1_SoftSeg  X2_DA_Content  X3_HardSeg  X4_R_Ratio  X5_Crosslink  \
0      0.6651            0.0      0.3349      1.3333        0.0000   
1      0.6353            0.0      0.3647      1.3846        0.0000   
2      0.6082            0.0      0.3918      1.4286        0.0000   
3      0.5389            0.0      0.4249      1.0000        0.0002   
4      0.7487            0.0      0.2513      1.0000        0.0000   

   DA_strategy  cross_class  Polyol_Type  Polyol_Mw_Score  Soft_Cryst  \
0            0            1          0.0              0.0         0.5   
1            0            1          0.0              0.0         0.5   
2            0            1          0.0      

In [9]:
def generate_virtual_library_final():
    print("正在根据 Final_Features_12.7.csv 结构生成虚拟配方库...")

    # === 1. 定义原材料基因库 (Design Space) ===
    # 这里的数值基于你 CSV 的统计范围和常见化学逻辑
    ranges = {
        # 软段定义: [名称, Polyol_Type, 分子量(Mw), Soft_Cryst(0/0.5/1)]
        # Polyol_Type: 0=聚醚(PPG/PTMG), 1=聚酯(PCL/PBA)
        'Soft_Segments': [
            ('PTMG1000', 0.0, 1000, 1.0), 
            ('PTMG2000', 0.0, 2000, 1.0), 
            ('PTMG4000', 0.0, 4000, 1.0),
            ('PPG1000',  0.0, 1000, 0.0),
            ('PPG2000',  0.0, 2000, 0.0), 
            ('PCL1000',  1.0, 1000, 1.0),
            ('PCL2000',  1.0, 2000, 1.0),
            ('PBA1000',  1.0, 1000, 1.0),
            ('PBA2000',  1.0, 2000, 1.0)  
        ],

        # 硬段定义: [名称, Iso_Type, Hard_Symmetry(0/1)]
        # Iso_Type: 1=IPDI(不对称), 4=MDI(对称), 2=HDI(对称)
        'Hard_Segments': [
            ('IPDI', 1.0, 0), 
            ('MDI',  4.0, 1),
            ('HDI',  2.0, 1),
            ('TDI', 3.5, 0)
        ],

        # 连续变量网格 (基于 CSV 的 min-max 分布)
        # X1 (软段) 范围: 0.45 ~ 0.85
        'X1_Soft_Range': np.linspace(0.45, 0.85, 20),
        
        # X2 (动态键) 范围: 0.0 ~ 0.45 (注意：这是额外添加的动态成分或改性比例)
        'X2_DA_Range': np.linspace(0.0, 0.45, 10),

        # X4 (NCO/OH) 范围: 0.9 ~ 1.1 (常用范围)
        'X4_Ratio_Range': [0.95, 1.0, 1.05],

        # X5 (交联度) 范围
        'X5_Crosslink_Range': [0.0, 0.05], # 主要是线性，少量交联

        # 其他工艺参数 (固定或少量变化)
        'DA_strategy': [0, 1], # 动态键策略
        'cross_class': [0, 1]  # 交联类别
    }

    virtual_data = []

    # === 2. 遍历组合生成配方 ===
    for soft in ranges['Soft_Segments']:
        s_name, s_type, s_mw, s_cryst = soft
        
        for hard in ranges['Hard_Segments']:
            h_name, h_iso_type, h_sym = hard

            for x1 in ranges['X1_Soft_Range']:
                for x2 in ranges['X2_DA_Range']:
                    for x4 in ranges['X4_Ratio_Range']:
                        for x5 in ranges['X5_Crosslink_Range']:
                            for da_strat in ranges['DA_strategy']:
                                for cross_cls in ranges['cross_class']:

                                    # --- A. 基础质量守恒计算 ---
                                    # 数据显示 X1 + X3 ≈ 1.0 (软段+硬段=1，添加剂除外)
                                    x3 = 1.0 - x1
                                    
                                    # 物理约束检查
                                    if x3 < 0.1: continue # 硬段太少无法成型

                                    # --- B. 特征计算 (Feature Engineering) ---
                                    row = {}

                                    # 1. 基础组分特征
                                    row['X1_SoftSeg'] = x1
                                    row['X2_DA_Content'] = x2
                                    row['X3_HardSeg'] = x3
                                    row['X4_R_Ratio'] = x4
                                    row['X5_Crosslink'] = x5

                                    # 2. 类别与结构特征
                                    row['Iso_Type'] = h_iso_type
                                    row['Hard_Symmetry'] = h_sym
                                    row['Polyol_Type'] = s_type
                                    
                                    # Polyol_Mw_Score: 你的数据里主要是 0 或 0.5 (可能归一化过或分段)
                                    # 假设逻辑: Mw < 2000 -> 0, Mw >= 2000 -> 0.5
                                    row['Polyol_Mw_Score'] = 0.5 if s_mw >= 2000 else 0.0

                                    row['Soft_Cryst'] = s_cryst
                                    row['DA_strategy'] = da_strat
                                    row['cross_class'] = cross_cls

                                    # 3. 关键交互特征 (Interaction Terms) - 必须与训练数据一致
                                    # Interact_Cryst_Content = X1 * Soft_Cryst
                                    row['Interact_Cryst_Content'] = x1 * s_cryst
                                    
                                    # Synergy_Feature = X1 * Soft_Cryst * Hard_Symmetry
                                    row['Synergy_Feature'] = x1 * s_cryst * h_sym

                                    # 4. 固定工艺参数 (设置为你实验的基准值)
                                    row['poly_tem'] = 80       # 聚合温度
                                    row['healing_temperature'] = 60.0 # 愈合测试温度
                                    row['healing_time'] = 24.0        # 愈合时间
                                    row['strain_rate'] = 50           # 拉伸速率
                                    
                                    # 辅助列 (方便筛选后查看配方)
                                    row['Recipe_Name'] = f"{s_name}_{h_name}_X1={x1:.2f}_DA={x2:.2f}"
                                    row['Soft_Mw_Raw'] = s_mw # 用于后续筛选分析
                                    row['Soft_Mw'] = s_mw  # 必须叫 Soft_Mw，和训练集一致
                                    row['Constraint_Factor'] = x3 / s_mw if s_mw > 0 else 0

                                    virtual_data.append(row)

    df_virtual = pd.DataFrame(virtual_data)
    return df_virtual
    

df_virtual = generate_virtual_library_final() # 忽略原来的 feature_cols 返回，我们下面重新定义
print(f"虚拟配方库生成完毕，共 {len(df_virtual)} 条数据")

正在根据 Final_Features_12.7.csv 结构生成虚拟配方库...
虚拟配方库生成完毕，共 172800 条数据


In [13]:
# 最优模型
# 1. 拉伸强度 (Strength) - GPR
kernel= 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-2,1e2), nu=2.5)+ WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5,1e1))
best_model_strength = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True, random_state=42)

# 2. 断裂伸长率 (Elongation) - GPR 
kernel = C(1.0, (1e-5, 1e5)) * \
         Matern(length_scale=1.0, length_scale_bounds=(1e-3, 1e4), nu=2.5) + \
         WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5, 1e1))
best_model_elongation = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True, random_state=42)

# 3. 自愈合 (Healing) - RF
best_model_healing = RandomForestRegressor(n_estimators=500, max_depth=30,min_samples_split=5,   random_state=42,n_jobs=-1)

# 训练
features_for_str = features_mech + ['Constraint_Factor']

# 1.强度
data_str = df.dropna(subset=['tensile_strength'] + features_for_str).copy()

X_str = data_str[features_for_str]
y_str = data_str['tensile_strength']

best_model_strength.fit(X_str, y_str)
print(f"强度模型训练完成。有效样本数：{len(X_str)}")

r2, rmse, _ = evaluate_model(best_model_strength, X_str, y_str, "Strength")
print(f"Strength | R2={r2:.3f}, RMSE={rmse:.2f}")


# 2.拉伸率
data_elo = df.dropna(subset=['elongation'] + features_for_str).copy()

X_elo = data_elo[features_for_str]
y_elo = data_elo['elongation']

best_model_elongation.fit(X_elo, y_elo)
print(f"伸长率模型训练完成。有效样本数: {len(X_elo)}")

r2, rmse, _ = evaluate_model(best_model_elongation, X_elo, y_elo, "elongation")
print(f"elongation | R2={r2:.3f}, RMSE={rmse:.2f}")

#3.自愈合率
data_heal = df.dropna(subset=['healing_eff'] + features_final).copy()

X_heal = data_heal[features_final]
y_heal = data_heal['healing_eff']

best_model_healing.fit(X_heal, y_heal)
print(f"自愈合模型训练完成。有效样本数: {len(X_heal)}")

r2, rmse, _ = evaluate_model(best_model_healing, X_heal, y_heal, "healing_eff")
print(f"healing_eff | R2={r2:.3f}, RMSE={rmse:.2f}")

强度模型训练完成。有效样本数：88
Strength | R2=0.611, RMSE=8.79
伸长率模型训练完成。有效样本数: 87
elongation | R2=0.404, RMSE=227.71
自愈合模型训练完成。有效样本数: 45
healing_eff | R2=0.594, RMSE=8.41


In [20]:
# === 3. 预测与筛选 (特征顺序严格修正版) ===

# A. 基础力学特征 (对应训练时的 features_mech)
# 注意：这里千万不要包含 'Constraint_Factor'，因为它在训练时是在最后才加上的
base_mech_features = [
    'X1_SoftSeg', 'X2_DA_Content', 'X3_HardSeg', 'X4_R_Ratio', 'X5_Crosslink',
    'DA_strategy', 'cross_class',
    'Polyol_Type', 'Polyol_Mw_Score', 'Soft_Cryst',
    'Iso_Type', 'Hard_Symmetry',
    'Interact_Cryst_Content', 'Synergy_Feature',
    'Soft_Mw'
]

# B. 严格重现训练时的拼接顺序

# 1. 强度/伸长率模型的特征顺序
# (假设你训练强度/伸长率时，只用了 base_mech_features 或者是 base + Constraint_Factor)
# 如果你之前的训练代码是 X_str = df[features_mech + ['Constraint_Factor']]，那就用下面这行：
pred_features_str_elo = base_mech_features + ['Constraint_Factor']
# 如果你训练强度时没用 Constraint_Factor，请把上面改成 pred_features_str_elo = base_mech_features

# 2. 自愈合模型的特征顺序 (必须与 features_final = features_heal + ['Constraint_Factor'] 一致)
# 训练逻辑回顾: features_mech -> ['healing_temperature', 'healing_time'] -> ['Constraint_Factor']
pred_features_heal = base_mech_features + ['healing_temperature', 'healing_time'] + ['Constraint_Factor']

# C. 准备预测数据
try:
    # 虚拟库补全默认工艺参数 (如果缺失)
    if 'healing_temperature' not in df_virtual.columns:
        df_virtual['healing_temperature'] = 60.0
    if 'healing_time' not in df_virtual.columns:
        df_virtual['healing_time'] = 24.0
        
    X_input_str_elo = df_virtual[pred_features_str_elo]
    X_input_heal = df_virtual[pred_features_heal]
    
except KeyError as e:
    print(f"列名缺失错误: {e}")
    # 打印一下当前的列名帮你debug
    print("当前虚拟库的列名:", df_virtual.columns.tolist())
    raise

# D. 执行预测
print("正在对虚拟库进行全量预测...")

# 预测强度
df_virtual['Pred_Strength'] = best_model_strength.predict(X_input_str_elo)

# 预测伸长率
df_virtual['Pred_Elongation'] = best_model_elongation.predict(X_input_str_elo)

# 预测自愈合
df_virtual['Pred_Healing'] = best_model_healing.predict(X_input_heal)

# 查看预测结果的统计分布
print("=== 预测值统计概览 ===")
print(df_virtual[['Pred_Strength', 'Pred_Healing', 'Pred_Elongation']].describe())

# 看看如果不加任何限制，最硬的配方能到多少？
print("\n=== 最硬配方 Top 3 ===")
print(df_virtual.sort_values(by='Pred_Strength', ascending=False)[
    ['Recipe_Name', 'Pred_Strength', 'Pred_Healing', 'Pred_Elongation']
].head(3))

# 看看如果不加任何限制，最“死”（不愈合）的配方是多少？
print("\n=== 最不愈合配方 Top 3 (Constraint Factor 验证) ===")
print(df_virtual.sort_values(by='Pred_Healing', ascending=True)[
    ['Recipe_Name', 'Pred_Strength', 'Pred_Healing', 'Pred_Elongation']
].head(3))


# === 4. 筛选 (策略保持不变) ===

# 策略 A: 刚性锚点
target_A = df_virtual[
    (df_virtual['Pred_Strength'] > 30) &      
    (df_virtual['Pred_Healing'] < 60) &       
    (df_virtual['Pred_Elongation'] > 100)     
].sort_values(by='Pred_Strength', ascending=False)

# 策略 B: 动态流体
target_B = df_virtual[
    (df_virtual['Pred_Healing'] > 80) &       
    (df_virtual['Pred_Strength'] > 5) &       
    (df_virtual['Pred_Strength'] < 20)        
].sort_values(by='Pred_Healing', ascending=False)

# === 5. 展示 ===
cols_to_show = ['Recipe_Name', 'X1_SoftSeg', 'X3_HardSeg', 'Soft_Mw', 'Constraint_Factor', 
                'Pred_Strength', 'Pred_Healing', 'Pred_Elongation']

print("\n====== Target A: 推荐的【刚性锚点】配方 (Top 5) ======")
print(target_A[cols_to_show].head(5))

print("\n====== Target B: 推荐的【动态流体】配方 (Top 5) ======")
print(target_B[cols_to_show].head(5))

target_A.head(5).to_csv('Target_A_Recipes.csv', index=False)
target_B.head(5).to_csv('Target_B_Recipes.csv', index=False)

正在对虚拟库进行全量预测...
=== 预测值统计概览 ===
       Pred_Strength   Pred_Healing  Pred_Elongation
count  172800.000000  172800.000000    172800.000000
mean       18.121260      76.063698       505.792881
std         0.653240      10.215548        37.042388
min         0.935627      55.117580       344.721780
25%        18.108499      67.535735       488.881753
50%        18.108523      76.894648       498.012305
75%        18.108526      86.464081       522.777779
max        31.176909      92.168902       746.675867

=== 最硬配方 Top 3 ===
                        Recipe_Name  Pred_Strength  Pred_Healing  \
24960  PTMG2000_MDI_X1=0.53_DA=0.00      31.176909     69.741506   
24720  PTMG2000_MDI_X1=0.51_DA=0.00      31.112383     69.695256   
24984  PTMG2000_MDI_X1=0.53_DA=0.05      30.600968     69.741506   

       Pred_Elongation  
24960       745.337656  
24720       742.471971  
24984       743.085044  

=== 最不愈合配方 Top 3 (Constraint Factor 验证) ===
                       Recipe_Name  Pred_Strength  Pr